## [5주차] 심화과제 - 다양한 형태의 입력을 가지는 LLM 서비스 개발

In [1]:
%pip install dotenv langchain openai chromadb pandas sentence-transformers matplotlib transformers accelerate torch

Note: you may need to restart the kernel to use updated packages.


### 데이터 선정

데이터는 공공API에서 제공하는 [국립중앙도서관 사서추천도서목록](https://www.nl.go.kr/NL/contents/N31101030900.do)을 사용한다.

xml 데이터를 json 데이터로 변환하였다.

In [2]:
import json

# JSON 데이터 로드
with open('library_books.json', 'r', encoding='utf-8') as f:
    books = json.load(f)

print("\n" + "="*50)
print("📚 데이터 구조")
print("="*50)
print("\n📍 데이터 키 목록:")
print(", ".join(list(books[0].keys())))

print("\n📍 첫 번째 도서 상세 정보:")
for key, value in books[0].items():
    print(f"\n{key}:")
    # 긴 텍스트는 줄바꿈하여 보기 좋게 출력
    if len(str(value)) > 100:
        # 100자마다 줄바꿈
        formatted_value = '\n'.join([str(value)[i:i+100] for i in range(0, len(str(value)), 100)])
        print(f"    {formatted_value}")
    else:
        print(f"    {value}")

print("\n" + "="*50)
print(f"📚 전체 도서 수: {len(books)}권")
print("="*50 + "\n")


📚 데이터 구조

📍 데이터 키 목록:
category_code, category_name, title, author, publisher, isbn, contents, table_of_contents, publish_year, recommend_year, recommend_month

📍 첫 번째 도서 상세 정보:

category_code:
    6

category_name:
    인문과학

title:
    제대로 연습하는 법 : 어학부터 스포츠까지, 인지심리학이 제시하는 배움의 기술

author:
    아투로 E. 허낸데즈 지음 ;방진이 옮김

publisher:
    북트리거 지학사

isbn:
    9791193378335

contents:
    한때 유행했던 일만 시간의 법칙을 기억하는가? 어떤 분야에서 전문가가 되려면 최소한 일만 시간의 훈련이 필요하다는 개념이다. 하지만 단순히 연습의 양이 많다고 해서 모두가 전문가가
 되는 것은 아니다. 이 책은 심리학자이자 다중언어 구사자, 테니스 선수로도 활약했던 저자가 학습과 훈련, 그리고 기량 향상의 상관관계를 연구한 결과를 담고 있다. 장 피아제, 노
엄 촘스키, 그리고 일만 시간의 법칙을 제창한 심리학자 안데르스 에릭손 등 대가들의 이론 및 최신 연구 결과를 바탕으로, 뇌과학과 인지심리학적 관점에서 '제대로 연습하는 법'을 탐
구한다. 아마추어 스포츠 선수, 유명 체스 선수, 다중언어 구사자, 피아노 연주자 등 다양한 사례 분석을 통해 연습의 물리적 양보다 중요한 것은 질적인 측면임을 강조한다. 적절한 
휴식 속에서 배운 것을 재조합하고, 몰입 상태에서 연습할 때 비로소 '최고'라는 목표에 도달할 수 있는 것이다. 벌써 2025년이 100일 넘게 흘렀다. 완연한 봄을 맞이하여 새로
운 기술을 배우고 싶거나, 그동안 노력에 비해 실력이 늘지 않는다고 느껴왔다면, 이 책을 길잡이 삼아 ‘제대로 연습하는 법’을 배워보면 어떨까?

table_of_contents:
    서론 : 작은 

category 정보를 확인한다.

In [3]:
from collections import defaultdict
import pandas as pd
from IPython.display import display

# 카테고리 코드와 이름을 매핑하여 저장
category_map = defaultdict(int)
for book in books:
    category_key = (book['category_code'], book['category_name'])
    category_map[category_key] += 1

# 데이터프레임으로 변환
df_categories = pd.DataFrame([
    {
        '분류 코드': code,
        '분류명': name,
        '도서 수': count
    }
    for (code, name), count in category_map.items()
])

# 도서 수를 기준으로 내림차순 정렬
df_categories = df_categories.sort_values('도서 수', ascending=False)

# 스타일 적용
styled_df = df_categories.style\
    .set_properties(**{'text-align': 'center'})\
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('background-color', '#f0f0f0')]},
        {'selector': 'td', 'props': [('text-align', 'center')]},
    ])\
    .format({'도서 수': '{:,d}'})  # 천 단위 구분자 추가

# 테이블 출력
print(f"\n📚 총 분류 수: {len(df_categories)}개\n")
display(styled_df)


📚 총 분류 수: 6개



,분류 코드,분류명,도서 수
0,6,인문과학,342
1,5,사회과학,338
3,4,자연과학,319
2,11,어문학,296
4,425,,92
5,8,,1


chuck size를 설정하기 위해 텍스트 길이를 파악했다.

임베딩할 데이터는 category_name, title, contents, table_of_contents이다.

In [4]:
import pandas as pd

# 각 도서별 텍스트 길이 계산
text_lengths = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    length_info = {
        '카테고리': len(book['category_name']),
        '제목': len(book['title']),
        '내용': len(book['contents']),
        '목차': len(book['table_of_contents']),
        '전체': len(text)
    }
    text_lengths.append(length_info)

# DataFrame으로 변환
df_lengths = pd.DataFrame(text_lengths)

# 기본 통계 계산
stats = df_lengths.agg(['min', 'max', 'mean', 'median', 'std']).round(2)

# 결과 출력
print("\n=== 각 필드별 텍스트 길이 통계 ===")
display(stats)

# 백분위수 계산
percentiles = df_lengths['전체'].quantile([0.25, 0.5, 0.75, 0.9])
print("\n=== 전체 텍스트 길이 백분위수 ===")
for p, v in percentiles.items():
    print(f"{int(p*100)}번째 백분위: {int(v)}자")


=== 각 필드별 텍스트 길이 통계 ===


,카테고리,제목,내용,목차,전체
min,0.00,1.00,135.00,0.00,323.00
max,4.00,75.00,861.00,6677.00,7296.00
mean,3.52,14.16,495.10,884.01,1417.79
median,4.00,11.00,490.00,734.50,1264.50
std,1.03,9.65,95.41,740.71,746.49



=== 전체 텍스트 길이 백분위수 ===
25번째 백분위: 881자
50번째 백분위: 1264자
75번째 백분위: 1758자
90번째 백분위: 2343자


### 백터 DB 생성

벡터 DB를 생성한다.

chuck size는 중앙값과 75% 사이의 값으로 설정하고, chunk overlap은 chunk size의 20% 정도로 설정한다.

임베딩 모델로는 무료로 사용할 수 있는 sentence-transformers의 다국어 지원 모델을 사용한다.

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch

# GPU 사용 가능 여부 확인 및 디바이스 설정
if torch.cuda.is_available():
    device = 'cuda'
    device_map = {'': 0}  # 단일 GPU 사용
elif torch.backends.mps.is_available():
    device = 'mps'
    device_map = {'': device}  # Apple Silicon GPU
else:
    device = 'cpu'
    device_map = {'': device}

print(f"Using device: {device}")

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': device}
)

# 텍스트 스플리터 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        # 중앙값과 75퍼센타일 사이의 값으로 설정
    chunk_overlap=300,      # chunk_size의 20% 정도로 설정
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]  # 문장 단위로 끊기도록 ". " 추가
)

# 임베딩할 텍스트 준비 및 메타데이터 포함
documents = []
for book in books:
    text = f"카테고리: {book['category_name']}\n제목: {book['title']}\n내용: {book['contents']}\n목차: {book['table_of_contents']}"
    
    # 메타데이터 준비
    metadata = {
        'title': book['title'],
        'category': book['category_name'],
        'author': book['author'],
        'isbn': book['isbn']
    }
    
    # 텍스트 분할 및 메타데이터 추가
    chunks = text_splitter.create_documents(
        texts=[text],
        metadatas=[metadata]
    )
    documents.extend(chunks)

# 벡터 DB 생성
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

Using device: mps


/var/folders/sm/f3jq7h1d5ms4gybg0zfs83fm0000gn/T/ipykernel_62894/3239963243.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


### 모델 공통 설정

프롬프트에 기본으로 사용할 템플릿을 정의한다.

In [6]:
template = """다음은 검색된 도서 정보들입니다:

{context}

이 정보들을 바탕으로 다음 질문에 답변해주세요: {question}

답변 작성 가이드:
1. 제시된 도서들의 난이도, 내용, 특징을 비교 분석해주세요.
2. 질문자의 요구사항에 가장 적합한 도서를 선정해주세요.
3. 선정한 도서가 첫 번째로 제시된 도서가 아니더라도 괜찮습니다.

답변은 한국어로 작성해주시고, 추천하는 도서의 제목과 저자를 반드시 포함해주세요."""

백터 db에서 유사도 상위 몇 개를 선정할지 개수를 선언한다.

In [7]:
top_k = 3

### 모델1. Open LLM

모델은 HuggingFace에서 제공하는 open LLM인 Bllossom/llama-3.2-Korean-Bllossom-3B를 사용한다.

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "Bllossom/llama-3.2-Korean-Bllossom-3B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map=device_map
)

# pad_token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 모델에도 pad_token_id 설정
model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

vectorstore의 similarity_search를 사용하여 백터 데이터베이스에서 의미적 유사도 검색을 수행한다.

similarity_search는 입력된 질문을 동일한 임베딩 모델로 벡터화하고, 백터 db에 저장된 모든 문서 백터들과 코사인 유사도를 계산하여 가장 유사도가 높은 k개의 문서를 반환한다.

다음과 같이 메타데이터 기반 필터링도 가능하다.

```python
filtered_docs = vectorstore.similarity_search(
    "인공지능 입문서",
    k=3,
    filter={"category": "자연과학"}
)
```

그리고 open LLM 모델을 사용해 추천 이유에 대한 텍스트를 생성하여 응답한다.

In [12]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

def get_book_recommendation(question: str):
    # 관련 문서 검색
    docs = vectorstore.similarity_search(question, k=top_k)
    
    # 검색된 문서들의 내용을 하나의 문자열로 결합
    context = "\n\n".join([
        f"제목: {doc.metadata['title']}\n"
        f"저자: {doc.metadata['author']}\n"
        f"카테고리: {doc.metadata['category']}\n"
        f"내용: {doc.page_content}"
        for doc in docs
    ])
    
    # 프롬프트 생성
    messages = [
        {"role": "user", "content": prompt.format(context=context, question=question)}
    ]
    
    # 입력 인코딩 및 attention mask 생성
    encoded = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    )
    
    input_ids = encoded.to(model.device)
    attention_mask = torch.ones_like(input_ids).to(model.device)
    
    # pad_token_id 설정
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
    
    # 종료 토큰 설정
    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # pad_token_id가 설정되어 있는지 확인하고 설정
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id
        model.config.pad_token_id = tokenizer.eos_token_id
    
    # 응답 생성
    outputs = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=terminators,
        # True: 확률적 샘플링 사용 (더 창의적인 응답)
        # False: 최대 확률 토큰 선택 (더 일관적인 응답)
        do_sample=True,
        temperature=0.6,
        # 누적 확률이 이 값을 넘는 토큰들만 고려
        # 높을수록 더 다양한 선택지 고려
        top_p=0.9,
    )
    
    response = tokenizer.decode(
        # 응답에는 입력(프롬프트)와 새로 생성된 결과가 모두 포함되어 있음
        # 입력 프롬프트를 제외하고 새로 생성된 부분만 출력
        outputs[0][input_ids.shape[-1]:], 
        skip_special_tokens=True
    )
    
    return {
        "answer": response,
        "similar_books": [
            {
                "title": doc.metadata["title"],
                "author": doc.metadata["author"],
                "category": doc.metadata["category"]
            }
            for doc in docs
        ]
    }

답변을 출력하기 위한 함수를 선언한다.

In [13]:
def format_long_text(text: str, width: int = 80):
    """긴 텍스트를 지정된 너비로 줄바꿈하여 반환"""
    words = text.split()
    lines = []
    current_line = []
    current_length = 0
    
    for word in words:
        word_length = len(word)
        if current_length + word_length + 1 <= width:
            current_line.append(word)
            current_length += word_length + 1
        else:
            lines.append(' '.join(current_line))
            current_line = [word]
            current_length = word_length + 1
    
    if current_line:
        lines.append(' '.join(current_line))
    
    return '\n'.join(lines)

def print_recommendation_result(result, question):
    """추천 결과를 포맷팅하여 출력"""
    print("\n" + "="*80)
    print("📚 도서 추천 결과")
    print("="*80)
    
    print("\n💭 질문:")
    print(f"{question}")
    
    print("\n📝 답변:")
    formatted_answer = format_long_text(result['answer'])
    print(f"{formatted_answer}")
    
    print(f"\n📚 검색된 유사 도서 (상위 {top_k}개):")
    for idx, book in enumerate(result['similar_books'], 1):
        print(f"\n{idx}. 제목: {book['title']}")
        print(f"   저자: {book['author']}")
        print(f"   카테고리: {book['category']}")
    
    print("\n" + "="*80)

질문을 추가해 모델을 테스트한다.

In [16]:
# 사용 예시
question = "인공지능 입문서 추천해주세요"
result = get_book_recommendation(question)

답변을 출력한다.

open LLM 모델은 프롬프트로 같이 입력한 책에 대한 정보를 문장 그대로 가져와서 사용하는 경우가 많았다.

응답도 오래걸렸다.

In [17]:
print_recommendation_result(result, question)


📚 도서 추천 결과

💭 질문:
인공지능 입문서 추천해주세요

📝 답변:
인공지능 입문서 추천을 위해 제시된 도서들의 내용을 비교 분석해보겠습니다. 1. **제목: 내 정원의 로봇** - **저자: 데보라 인스톨**
- **난이도 및 내용**: 이 책은 인공지능 소재에 작가의 상상력을 더해 풀어낸 따뜻한 소설입니다. 인공지능 하인이 집안일을 돕고,
안드로이드가 운전을 하는 미래 영국의 어느 마을에 사는 주인공을 중심으로 이야기가 전개됩니다. - **특징**: 인공지능의 다양한 성공 사례와
주인공의 성장 과정을 통해 인공지능의 가능성을 시사합니다. 주인공 벤은 낡고 고장난 로봇 '탱'을 고치기 위해 여행을 떠나고, 그 과정에서
둘은 다양한 사건을 겪으며 서로를 위로하고 성장합니다. 2. **제목: 앙코르 내 인생** - **저자: 신동흔, 김수혜, 김미리, 김신영**
- **난이도 및 내용**: 이 책은 14명의 주인공이 다양한 직업과 삶을 통해 성장하고 성숙하는 과정을 다룹니다. 각 주인공이 자신의 삶의
고장을 해결하고 성장하는 이야기를 통해 인공지능과 관련된 주제를 다룹니다. - **특징**: 다양한 직업과 삶의 경험을 통해 인공지능의
가능성을 시사하며, 주인공들이 성장하고 성숙하는 과정을 통해 인공지능에 대한 공감을 유도합니다. 3. **제목: 천개의 파랑** - **저자:
천선란** - **난이도 및 내용**: 이 책은 2035년을 배경으로 한 SF소설입니다. 인간과 로봇, 경주마가 함께 주인공을 이루며 다양한
이야기를 전개합니다. - **특징**: 과학기술의 발전으로 인해 소외된 인간과 동물, 로봇에 대해 이야기하며, 주인공들의 서로를 위한 서투른
배려와 성장 과정을 통해 인공지능에 대한 공감을 유도합니다. 이러한 도서들의 특징과 내용을 비교하면, **내 정원의 로봇**은 인공지능의
다양한 성공 사례와 주인공의 성장 과정을 통해 인공지능의 가능성을 시사하는 소설입니다. 반면, **앙코르 내 인생**은 다양한 직업과 삶의
경험을 통해 인공지능의 가능성을

"재미있는 소설 추천해줘"와 같은 추상적인 질문에도 답변하였지만, 벡터db에서 유사도로 추천된 도서 리스트가 아쉬웠다.

추천된 도서 리스트 중에는 비교적 질문과 가까운 도서를 추천하였다. 

In [18]:
question = "재미있는 소설 추천해줘"
result = get_book_recommendation(question)

In [19]:
print_recommendation_result(result, question)


📚 도서 추천 결과

💭 질문:
재미있는 소설 추천해줘

📝 답변:
제시된 도서들의 난이도, 내용, 특징을 비교 분석해보고, 질문자의 요구사항에 가장 적합한 도서를 선정해보겠습니다. 1. 제시된 도서들의
난이도와 내용을 분석하면 다음과 같습니다: - **옛그림 속 양반의 한평생**: 이 책은 조선시대 양반의 일상 생활을 그린 소설입니다.
조선시대에 살았던 평범한 양반들의 일상과 삶을 통해 조선시대 사회의 특성을 이해할 수 있는 작품입니다. 책의 내용은 단순한 일상 이야기보다는
조선시대 양반들의 삶을 깊이 있게 설명하고 있습니다. - **그 무렵 누군가**: 이 책은 히가시노 게이고 작가의 단편 소설을 모아 한 권으로
엮은 책입니다. 히가시노 작가의 작품은 극적 재미와 예리한 비판적 의식을 가지고 있으며, 이 책에서는 다양한 주제와 다채로운 소재를 다루고
있습니다. - **도시를 걷는 시간**: 이 책은 서울 시내 표석을 통해 조선시대와 현재의 차이를 비교하며 역사와 문화를 탐구한 작품입니다.
작가는 조선시대와 현재의 주변에 위치한 표석을 통해 시간의 길을 거슬러 역사를 단순히 과거로 치부할 것이 아닌 어제와 오늘 그리고 내일을
만나는 순간임을 깨닫고자 합니다. 2. 질문자의 요구사항을 분석해보고, 가장 적합한 도서를 선정해보겠습니다. - 질문자는 재미있는 소설을
추천해주고자 합니다. 이 경우, **그 무렵 누군가**와 **도시를 걷는 시간**이 모두 재미있는 소설이 될 수 있지만, 질문자가 단순히
재미있는 소설을 찾는 것이라면, **그 무렵 누군가**가 더 적합할 수 있습니다. 이 책은 히가시노 게이고 작가의 작품을 통해 다양한 주제와
소재를 다루고 있으며, 극적 재미와 예리한 비판적 의식을 가지고 있어 독자들에게 큰 호기심을 자극할 수 있습니다. 3. 선정한 도서의 제목과
저자는 다음과 같습니다: - **그 무렵 누군가**: 히가시노 게이고 저자 이 책은 히가시노 게이고 작가의 작품을 통해 다양한 주제와 소재를
다루고 있으며, 극적 재미와 예리한 비판적 의

### 모델2: GPT
gpt 모델로는 gpt-4o-mini를 사용한다.

API KEY를 불러온다.

In [23]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

모델은 gpt-4o-mini를 사용한다.

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,          # 적당한 창의성
    top_p=0.9,               # 다양한 선택 허용
    max_tokens=512,          # 적당한 길이
    presence_penalty=0.2,    # 약간의 새로운 관점 도입
    frequency_penalty=0.3,   # 반복 방지
    api_key=api_key
)

In [25]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template
)

def get_book_recommendation_from_gpt(question: str):
    # 관련 문서 검색
    docs = vectorstore.similarity_search(question, k=top_k)
    
    # 검색된 문서들의 내용을 하나의 문자열로 결합
    context = "\n\n".join([
        f"제목: {doc.metadata['title']}\n"
        f"저자: {doc.metadata['author']}\n"
        f"카테고리: {doc.metadata['category']}\n"
        f"내용: {doc.contents}"
        for doc in docs
    ])
    
    # 프롬프트 생성
    formatted_prompt = prompt.format(
        context=context,
        question=question
    )
    
    # LLM에 질의
    response = llm.invoke(formatted_prompt)
    
    return {
        "answer": response.content,
        "similar_books": [
            {
                "title": doc.metadata["title"],
                "author": doc.metadata["author"],
                "category": doc.metadata["category"]
            }
            for doc in docs
        ]
    }

In [26]:
# 사용 예시
question = "인공지능 입문서 추천해주세요"
result = get_book_recommendation_from_gpt(question)

결과를 출력한다.

open LLM에 비해 input으로 넣은 도서 정보 문장을 그대로 가져오기 보다는 요약을 하고 있고, 질문에 적합한지에 대해서도 더 추론하여 답변한다.

In [27]:
print_recommendation_result(result, question)


📚 도서 추천 결과

💭 질문:
인공지능 입문서 추천해주세요

📝 답변:
제시된 도서 정보들을 바탕으로 인공지능 입문서를 추천해 드리겠습니다. 1. **도서 비교 분석**: - **내 정원의 로봇 (저자: 데보라
인스톨)**: 이 책은 인공지능을 소재로 한 소설로, 주인공 벤이 고장 난 로봇 탱을 수리하며 겪는 성장 이야기를 다룹니다. 인공지능의 발전과
인간의 감정을 연결지어 따뜻한 이야기로 풀어내고 있어, 인공지능 기술에 대한 직접적인 설명보다는 그로 인해 변화하는 인간 관계와 정서를
중심으로 하고 있습니다. - **앙코르 내 인생 (저자: 신동흔 외)**: 이 책은 다양한 사람들의 인생 이야기를 통해 새로운 삶의 전환점을
탐구하는 에세이집입니다. 주제는 삶의 변화와 도전이지만, 인공지능 관련 내용은 포함되어 있지 않습니다. - **천개의 파랑 (저자:
천선란)**: 이 SF 소설은 2035년의 미래를 배경으로 하며, 경주마와 로봇 기수 간의 관계를 통해 인간과 동물, 로봇에 대한 사회적
이슈를 다루고 있습니다. 역시 인공지능에 대한 기술적 설명보다는 감정적인 서사와 사회적 메시지를 중심으로 전개됩니다. 2. **요구사항에
적합한 도서 선정**: - 제시된 도서들 중에서 인공지능에 대한 깊이 있는 이해를 원하는 입문서로는 적합하지 않습니다. 하지만, '내 정원의
로봇'은 인공지능과 관련된 주제를 다루고 있으며, 이를 통해 독자는 인공지능의 적용 가능성과 인간과의 상호작용을 간접적으로 경험할 수
있습니다. 3. **추천 도서**: - 따라서 인공지능 입문서로는 **‘내 정원의 로봇’ (저자: 데보라 인스톨)**을 추천드립니다. 이 책은
소설 형식으로 이야기되기 때문에 부담 없이 읽을 수 있으며, 현대 사회에서의 인공지능에 대한 다양한 시각을 제공받을 수 있습니다. 읽으시는 데
도움이 되길 바랍니다!

📚 검색된 유사 도서 (상위 3개):

1. 제목: 내 정원의 로봇
   저자: 데보라 인스톨
   카테고리: 어문학

2. 제목: 앙코르 내 인생
   저자:

"재미있는 소설 추천해줘"와 같은 추상적인 질문에도 답변하였지만, 벡터db에서 유사도로 추천된 도서 리스트가 아쉬웠다.

추천된 도서 리스트 중에는 비교적 질문과 가까운 도서를 추천하였다. 

output 길이 제한으로 result가 중간에 중단되었다.

In [29]:
question = "재미있는 소설 추천해줘"
result = get_book_recommendation_from_gpt(question)

In [30]:
print_recommendation_result(result, question)


📚 도서 추천 결과

💭 질문:
재미있는 소설 추천해줘

📝 답변:
재미있는 소설을 추천하기 위해 제시된 도서들의 난이도, 내용, 특징을 비교 분석해보겠습니다. 1. **도서 비교 분석**: - **『옛그림 속
양반의 한평생』 - 허인욱**: 이 책은 인문과학 분야로, 조선시대 양반들의 삶을 풍속화와 함께 설명하는 내용입니다. 역사적 사실과 사회적
맥락에 대한 깊은 이해를 제공하지만, 소설이 아닌 비소설이기 때문에 이야기의 흡입력이나 긴장감은 상대적으로 낮습니다. - **『그 무렵
누군가』 - 히가시노 게이고**: 일본의 베스트셀러 작가 히가시노 게이고의 단편소설 모음집으로, 다양한 주제와 서스펜스를 담고 있습니다. 특히
'아빠, 안녕'과 같은 몽환적인 소재와 '수수께끼가 가득'처럼 사회적 비판을 담은 이야기가 있어 흥미진진합니다. 짧은 분량의 여러 이야기를
통해 긴장감과 재미를 느낄 수 있습니다. - **『도시를 걷는 시간』 - 김별아**: 어문학 장르로 서울 시내의 조선시대 표석을 탐방하며
과거와 현재를 연결하는 에세이입니다. 역사적 요소와 도시 탐방에 중점을 두고 있지만, 소설적인 이야기 전개나 스릴이 부족하여 재미를 중시하는
독자에게는 다소 지루할 수 있습니다. 2. **추천 도서 선정**: 질문자가 "재미있는 소설"을 요청한 만큼, 문학적 재미와 긴장감을 제공하는
작품이 필요합니다. 따라서 히가시노 게이고의 『그 무렵 누군가』가 가장 적합한 선택입니다. 이 책은 단편소설 모음으로 다양한 이야기들이
포함되어 있어 독자가 쉽게 접근하고 즐길 수 있으며, 작가 특유의 서스펜스와 극적 재미는 많은 독자들에게 호평받고 있습니다. 3. **추천
도서 정보**: - **제목**: 그 무렵 누군가 - **저자**: 히가시노 게이고 이 책을 통해 여름의 더

📚 검색된 유사 도서 (상위 3개):

1. 제목: 옛그림 속 양반의 한평생
   저자: 허인욱
   카테고리: 인문과학

2. 제목: 그 무렵 누군가
   저자: 히가시노 게이고 
   카테고리: 어문학

3